In [29]:
import numpy as np
import scipy.stats as scs
import matplotlib.pyplot as plt
import scipy.optimize as sco 

import kernels as kl

In [49]:
def k_gauss(x,y,s):
    return np.exp(-np.linalg.norm(x-y)**2/s**2)

In [57]:
alpha = 0.1

mux = np.array([5,5])
Lx = np.random.randn(2,2)
Sigmax = Lx @ Lx.transpose()
muy = np.array([0,0])
Ly =  np.random.randn(2,2)
Sigmay = Ly @ Ly.transpose() 

n = 50
m = 50

y = scs.multivariate_normal.rvs(muy,Sigmay,m)
x = scs.multivariate_normal.rvs(mux,Sigmax,n)


sigma = 5
k = lambda x,y :  kl.k_gauss(x,y,sigma)
kk = lambda x,y : k_gauss(x, y, sigma)



Ky = 1/m * k(y,y) 
Ly,V = np.linalg.eig(Ky)
V = V.transpose()
Ly = np.real(Ly)
Packy = [Ky,Ly,V]

In [58]:
def log_ou_0(t):
    return np.where(t > 0, np.log(t), 0.)

In [75]:
Kx = 1e-9 * np.identity(n) + 1/n * k(x,x)
    

Kxy = k(x,y) 
Lx,U = np.linalg.eig(Kx)
U = np.real(U).T
Lx = np.real(Lx)
Trxy = 0

K = 1e-9 * np.identity(n+m) + np.concatenate([np.concatenate([alpha * Kx, np.sqrt(alpha*(1-alpha)/(n*m)) *Kxy],axis = 1),np.concatenate([np.sqrt(alpha*(1-alpha)/(n*m)) *Kxy.transpose(),(1-alpha)* Ky],axis = 1)],axis = 0)
Lz,W = np.linalg.eig(K)
W = W.T

A = np.concatenate([np.concatenate([1/alpha * np.identity(n), np.zeros((n,m))],axis = 1),np.concatenate([np.zeros((m,n)),np.zeros((m,m))],axis = 1)],axis = 0)

Trxx = np.sum(Lx * log_ou_0(Lx))
Trxy = np.trace(A @ W.T @ np.diag(Lz * log_ou_0(Lz)) @ W)

In [76]:
M = np.zeros((n+m,n+m))
for j in range(n+m):
    M = M - np.linalg.norm(W[j][:n]**2)/(Lz[j]**2) * W[j][:,None] @ W[j][None,:]
    for l in range(n+m):
        if k != j:
            M = M + (np.log(Lz[j])/(Lz[l]-Lz[j] + 1e-9)**2 + 1/(Lz[j]*(Lz[l]-Lz[j] + 1e-9))) * np.dot(W[j][:n],W[l][:n]) * (W[j][:,None] @ W[l][None,:])
        #print("e")
w1 = x[1]
w2 = x[2]

T1 = np.concatenate([np.sqrt(alpha/n) * np.array([kk(w1,x[i]) for i in range(n)]),np.sqrt((1-alpha)/m) * np.array([kk(w1,y[j]) for j in range(m)])])
T2 = np.concatenate([np.sqrt(alpha/n) * np.array([kk(w2,x[i]) for i in range(n)]),np.sqrt((1-alpha)/m) * np.array([kk(w2,y[j]) for j in range(m)])])

In [77]:
S_inv = np.linalg.inv(Kx)

In [81]:
print(alpha*kk(w1,w2) * T1 @ M @ T1)
print(np.trace(S_inv))

(-2715106043972277.5-1.9936083579979782j)
33005543433.324207
